[![Open in Colab](https://colab.research.google.com/assets//colab-badge.svg)](https://colab.research.google.com/github/Joykw1/NLP_RAG_project/blob/main/Code/Multipassage_retrieval.ipynb)

In [1]:
!pip install torch transformers bitsandbytes accelerate outlines datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

checkpoint = "Qwen/Qwen2.5-1.5B-Instruct"


# Configure 8-bit quantization. We use this to save VRAM, as we don't have a lot available.
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True  # Enables 8-bit quantization
)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    quantization_config=bnb_config,  # Apply BitsAndBytesConfig
    device_map="cuda"   # Assign to GPU
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [64]:
from datasets import load_dataset


# sample 200 examples per language:
ru_dataset = load_dataset("JRQi/Global-MMLU-emb", "ru")
en_dataset = load_dataset("JRQi/Global-MMLU-emb", "en")
de_dataset = load_dataset("JRQi/Global-MMLU-emb", "de")

ru_dataset = ru_dataset["test"].shuffle(seed=42).select(range(250))
en_dataset = en_dataset["test"].shuffle(seed=42).select(range(250))
de_dataset = de_dataset["test"].shuffle(seed=42).select(range(250))

In [65]:
dataset_dict = {
    'ru': ru_dataset,
    'en': en_dataset,
    'de': de_dataset
}

In [66]:
en_dataset

Dataset({
    features: ['id', 'subject', 'subject_category', 'question', 'option_a', 'option_b', 'option_c', 'option_d', 'answer', 'emb'],
    num_rows: 250
})

In [53]:
import json

with open("retrieved_passages.json", "r") as f:
    retrieved_passages = json.load(f)

In [54]:
for key in list(retrieved_passages)[:10]:
    # print(key, retrieved_passages[key])
    print(retrieved_passages[key].keys())

dict_keys(['en_crosslingual', 'en_multilingual', 'de_crosslingual', 'de_multilingual', 'ru_crosslingual', 'ru_multilingual'])
dict_keys(['en_crosslingual', 'en_multilingual', 'de_crosslingual', 'de_multilingual', 'ru_crosslingual', 'ru_multilingual'])
dict_keys(['en_crosslingual', 'en_multilingual', 'de_crosslingual', 'de_multilingual', 'ru_crosslingual', 'ru_multilingual'])
dict_keys(['en_crosslingual', 'en_multilingual', 'de_crosslingual', 'de_multilingual', 'ru_crosslingual', 'ru_multilingual'])
dict_keys(['en_crosslingual', 'en_multilingual', 'de_crosslingual', 'de_multilingual', 'ru_crosslingual', 'ru_multilingual'])
dict_keys(['en_crosslingual', 'en_multilingual', 'de_crosslingual', 'de_multilingual', 'ru_crosslingual', 'ru_multilingual'])
dict_keys(['en_crosslingual', 'en_multilingual', 'de_crosslingual', 'de_multilingual', 'ru_crosslingual', 'ru_multilingual'])
dict_keys(['en_crosslingual', 'en_multilingual', 'de_crosslingual', 'de_multilingual', 'ru_crosslingual', 'ru_multilin

In [68]:
import pandas as pd
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import json
from tqdm.notebook import tqdm
import os

# Define the configurations
CONFIGS = {
    "ru_multilingual": {
        "question_lang": "ru",
        "context_langs": ["ru"],
        "answer_lang": "ru",
        "base_dataset": "ru"
    },
    "en_multilingual": {
        "question_lang": "en",
        "context_langs": ["en"],
        "answer_lang": "en",
        "base_dataset": "en"
    },
    "de_multilingual": {
        "question_lang": "de",
        "context_langs": ["de"],
        "answer_lang": "de",
        "base_dataset": "de"
    },
    "ru_crosslingual": {
        "question_lang": "ru",
        "context_langs": ["ru", "en", "de"],
        "answer_lang": "ru",
        "base_dataset": "ru"
    },
    "en_crosslingual": {
        "question_lang": "en",
        "context_langs": ["ru", "en", "de"],
        "answer_lang": "en",
        "base_dataset": "en"
    },
    "de_crosslingual": {
        "question_lang": "de",
        "context_langs": ["ru", "en", "de"],
        "answer_lang": "de",
        "base_dataset": "de"
    }
}

# Choose which configuration to run
for config_name in CONFIGS:
    config = CONFIGS[config_name]

    print(f"Running with configuration: {config_name}")
    print(f"Question language: {config['question_lang']}")
    print(f"Context languages: {config['context_langs']}")
    print(f"Answer language: {config['answer_lang']}")

    # Parameters
    batch_size = 10        # How often to save to CSV
    max_samples = None     # Set to a number to limit processing (e.g., 10 for testing)


    # Create output file paths
    output_path = f"multilingual_qa_results_{config_name}.jsonl"
    output_csv = f"multilingual_qa_results_{config_name}.csv"


    # Function to create a chat template based on the configuration
    def create_chat_input(question, contexts, options, config):
        # Get configuration parameters
        question_lang = config["question_lang"]
        context_langs = config["context_langs"]

        # Map language codes to full names
        lang_names = {"ru": "Russian", "en": "English", "de": "German"}

        # Build the context text
        contexts_text = ""
        for passage in contexts:
            lang_name = lang_names[passage[0]]
            context = passage[1]
            contexts_text += f"{lang_name} Context: {context}\n\n"

        # Build the answers text
        options_text = ""
        for option in options:
            options_text += f"{option[0]}: {option[1]}\n"

        # Get the name of the question language
        question_lang_name = lang_names[question_lang]

        # Create the prompt with appropriate language instruction
        prompt = f"""
    I need help with a question answering task using multiple languages.

    {contexts_text}Question ({question_lang_name}): {question}

    Answers:
    {options_text}

    Your answer must be chosen from the given answers.
    Your answer should ALWAYS be the letter of the correct answer (so it should be in either A, B, C or D, with no further explanation).
    NEVER deviate from this output format.
    """
        return prompt

    # Function to generate an answer
    def generate_answer(input_text):
        # Apply the model's chat template
        if tokenizer.chat_template:
            messages = [{"role": "user", "content": input_text}]
            formatted_input = tokenizer.apply_chat_template(messages, tokenize=False)
        else:
            # Simple fallback if no chat template
            formatted_input = f"<|im_start|>user\n{input_text}<|im_end|>\n<|im_start|>assistant\n"

        # Tokenize and generate
        inputs = tokenizer(formatted_input, return_tensors="pt").to("cuda")

        with torch.no_grad():
            outputs = model.generate(
                inputs.input_ids,
                max_new_tokens=100,
                temperature=0.7,
                top_p=0.9,
                do_sample=True
            )

        response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
        response = response.strip()

        # Clean up role prefixes that might appear in the output
        # Remove "user" or "system" prefixes that might appear at the beginning
        response = re.sub(r'^(user|system)\s+', '', response)

        # Clean up any remaining instances within the text
        response = re.sub(r'\n(user|system)\s+', '\n', response)

        return response

    base_dataset = dataset_dict[CONFIGS[config_name]['base_dataset']]

    existing_ids = set()
    results = []
    total_examples = len(base_dataset)
    if max_samples is not None:
        total_examples = min(total_examples, max_samples)

    for idx, entry in enumerate(list(base_dataset)):
        question = entry['question']
        id = entry['id']

        if id in existing_ids:
            print(f"Skipping already processed example ID: {id}")
            continue
        contexts = retrieved_passages[entry['id']][config_name]
        original_answer = entry['answer']
        options = [["A", entry['option_a']], ["B", entry['option_b']], ["C", entry['option_c']], ["D", entry['option_d']]]

        prompt = create_chat_input(question, contexts, options, config)

        # provide the llm with 3 opportunities to answer in the correct format, else continue
        for i in range(3):
            answer = generate_answer(prompt)
            if answer in ["A", "B", "C", "D"]:
                break
        print(generate_answer(prompt))
        print(f"Original Answer: {original_answer}")

        existing_ids.add(id)

        # Create result row
        result = {
            'id': id,
            f'question_{config["question_lang"]}': question,
            'original_answer': original_answer,
            'model_answer': answer
        }

        # Save to list
        results.append(result)

        # Save incrementally to file
        with open(output_path, 'a', encoding='utf-8') as f:
            f.write(json.dumps(result, ensure_ascii=False) + '\n')

        # Create/update CSV file after each batch
        if len(results) % batch_size == 0 or idx == total_examples - 1:
            pd.DataFrame(results).to_csv(output_csv, index=False)
            print(f"Saved {len(results)} results to CSV")


    # Final DataFrame
    final_df = pd.DataFrame(results)
    print(f"Total processed examples: {len(final_df)}")

    # Display the first few rows
    if len(final_df) > 0:
        print("\nSample results:")
        display(final_df.head())

Running with configuration: ru_multilingual
Question language: ru
Context languages: ['ru']
Answer language: ru
D
Original Answer: A
B
Original Answer: B
C
Original Answer: C
A
Original Answer: C
B
Original Answer: A
C
Original Answer: C
B
Original Answer: B
A
Original Answer: C
A
Original Answer: C
C
Original Answer: B
Saved 10 results to CSV
A
Original Answer: A
A
Original Answer: C
D
Original Answer: D
A
Original Answer: B
B
Original Answer: B
B
Original Answer: A
B
Original Answer: C
B
Original Answer: C
subject
C
Original Answer: C
C
Original Answer: C
Saved 20 results to CSV
D
Original Answer: C
A
Original Answer: B
A
Original Answer: A
B
Original Answer: D
A
Original Answer: D
A
Original Answer: A
A
Original Answer: B
C
Original Answer: A
A
Original Answer: A
A
Original Answer: C
Saved 30 results to CSV
B
Original Answer: B
B
Original Answer: C
A
Original Answer: D
subject
C
Original Answer: C
B
Original Answer: D
D
Original Answer: D
subject
A
Original Answer: D
B
Original Answ

,id,question_ru,original_answer,model_answer
0,college_physics/test/100,"Позитроний – это атом, образованный электроном...",A,B
1,anatomy/test/22,Региональный лимфатический дренаж левой сторон...,B,B
2,high_school_statistics/test/123,Совет школы крупного школьного округа предлага...,C,C
3,miscellaneous/test/166,Какое звание дает право генералу армии США нос...,C,C
4,prehistory/test/140,Что НЕ было решающим фактором в развитии месоп...,A,B


Running with configuration: en_multilingual
Question language: en
Context languages: ['en']
Answer language: en
B
Original Answer: A
subject
A
Original Answer: B
C
Original Answer: C
D
Original Answer: C
A
Original Answer: A
A
Original Answer: C
A
Original Answer: B
C
Original Answer: C
B
Original Answer: C
A
Original Answer: B
Saved 10 results to CSV
A
Original Answer: A
A
Original Answer: C
D: codified
Original Answer: D
D
Original Answer: B
B
Original Answer: B
A
Original Answer: A
A
Original Answer: C
C
Original Answer: C
A
Original Answer: C
C
Original Answer: C
Saved 20 results to CSV
D
Original Answer: C
D
Original Answer: B
A
Original Answer: A
B
Original Answer: D
D: rational emotive
Original Answer: D
A
Original Answer: A
A
Original Answer: B
A
Original Answer: A
subject
A
Original Answer: A
C
Original Answer: C
Saved 30 results to CSV
B
Original Answer: B
B
Original Answer: C
B
Original Answer: D
A
Original Answer: C
D
Original Answer: D
D
Original Answer: D
A
Original Answe

,id,question_en,original_answer,model_answer
0,college_physics/test/100,Positronium is an atom formed by an electron a...,A,B
1,anatomy/test/22,The regional lymphatic drainage of the left si...,B,C
2,high_school_statistics/test/123,A school board of a large school district is p...,C,C
3,miscellaneous/test/166,What rank entitles a general in the US Army to...,C,D
4,prehistory/test/140,What was NOT a deciding factor in the developm...,A,A


Running with configuration: de_multilingual
Question language: de
Context languages: ['de']
Answer language: de
B
Original Answer: A
B
Original Answer: B
C
Original Answer: C
C
Original Answer: C
C
Original Answer: A
A
Original Answer: C
A
Original Answer: B
subject
A
Original Answer: C
B
Original Answer: C
D
Original Answer: B
Saved 10 results to CSV
B
Original Answer: A
A
Original Answer: C
D
Original Answer: D
A
Original Answer: B
C
Original Answer: B
A
Original Answer: A
ystem
A
Original Answer: C
B
Original Answer: C
A
Original Answer: C
C
Original Answer: C
Saved 20 results to CSV
A
Original Answer: C
A
Original Answer: B
A
Original Answer: A
B
Original Answer: D
A
Original Answer: D
A
Original Answer: A
A
Original Answer: B
C
Original Answer: A
A
Original Answer: A
C
Original Answer: C
Saved 30 results to CSV
B
Original Answer: B
C
Original Answer: C
D
Original Answer: D
A
Original Answer: C
B
Original Answer: D
A
Original Answer: D
B
Original Answer: D
D
Original Answer: B
B
Or

...

,id,question_ru,original_answer,model_answer
0,college_physics/test/100,"Позитроний – это атом, образованный электроном...",A,B
1,anatomy/test/22,Региональный лимфатический дренаж левой сторон...,B,B
2,high_school_statistics/test/123,Совет школы крупного школьного округа предлага...,C,C
3,miscellaneous/test/166,Какое звание дает право генералу армии США нос...,C,A
4,prehistory/test/140,Что НЕ было решающим фактором в развитии месоп...,A,B


Running with configuration: en_crosslingual
Question language: en
Context languages: ['ru', 'en', 'de']
Answer language: en
B
Original Answer: A
A
Original Answer: B
C
Original Answer: C
D
Original Answer: C
ystem
A
Original Answer: A
A
Original Answer: C
B
Original Answer: B
C
Original Answer: C
B
Original Answer: C
A
Original Answer: B
Saved 10 results to CSV
A
Original Answer: A
A
Original Answer: C
D
Original Answer: D
D
Original Answer: B
B
Original Answer: B
A
Original Answer: A
A
Original Answer: C
C
Original Answer: C
D
Original Answer: C
C
Original Answer: C
Saved 20 results to CSV
D
Original Answer: C
D
Original Answer: B
A
Original Answer: A
A
Original Answer: D
D
Original Answer: D
A
Original Answer: A
A
Original Answer: B
A
Original Answer: A
A
Original Answer: A
C
Original Answer: C
Saved 30 results to CSV
B
Original Answer: B
B
Original Answer: C
B
Original Answer: D
A
Original Answer: C
A
Original Answer: D
D
Original Answer: D
A
Original Answer: D
D
Original Answer: B


,id,question_en,original_answer,model_answer
0,college_physics/test/100,Positronium is an atom formed by an electron a...,A,B
1,anatomy/test/22,The regional lymphatic drainage of the left si...,B,C
2,high_school_statistics/test/123,A school board of a large school district is p...,C,C
3,miscellaneous/test/166,What rank entitles a general in the US Army to...,C,D
4,prehistory/test/140,What was NOT a deciding factor in the developm...,A,A


Running with configuration: de_crosslingual
Question language: de
Context languages: ['ru', 'en', 'de']
Answer language: de
B
Original Answer: A
B
Original Answer: B
C
Original Answer: C
subject
C
Original Answer: C
B
Original Answer: A
subject
A
Original Answer: C
B
Original Answer: B
C
Original Answer: C
B
Original Answer: C
A
Original Answer: B
Saved 10 results to CSV
B
Original Answer: A
A
Original Answer: C
D
Original Answer: D
A
Original Answer: B
C
Original Answer: B
A
Original Answer: A
A
Original Answer: C
B
Original Answer: C
C
Original Answer: C
C
Original Answer: C
Saved 20 results to CSV
A
Original Answer: C
D
Original Answer: B
A
Original Answer: A
B
Original Answer: D
A
Original Answer: D
A
Original Answer: A
ystem
A
Original Answer: B
B
Original Answer: A
A
Original Answer: A
C
Original Answer: C
Saved 30 results to CSV
B
Original Answer: B
C
Original Answer: C
A
Original Answer: D
A
Original Answer: C
A
Original Answer: D
A
Original Answer: D
B
Original Answer: D
A
Ori

,id,question_de,original_answer,model_answer
0,college_physics/test/100,"Positronium ist ein Atom, das aus einem Elektr...",A,B
1,anatomy/test/22,Die regionale Lymphdrainage der linken Seite d...,B,B
2,high_school_statistics/test/123,Die Schulbehörde eines großen Schulbezirks möc...,C,C
3,miscellaneous/test/166,Welcher Rang befähigt einen General der US-Arm...,C,C
4,prehistory/test/140,Was war KEIN Entscheidungsfaktor für die Entwi...,A,A
